In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import os
import json
import dash
from pathlib import Path    
from moviepy.video.io.VideoFileClip import VideoFileClip

current_dir = Path.cwd()
project_root = current_dir.parent  # Move up one level
git_dir = project_root / "bachelor_vff"
data_dir = project_root / "vff_loc_data"
video_dir = project_root / "videos"


In [2]:
events_df = pd.read_csv(git_dir / "events_data_2.csv")
passes_df = events_df[events_df['optaEvent.typeId'] == 1]
print(passes_df[['optaEvent.timeMin', 'optaEvent.timeSec', 'optaEvent.playerName', 'optaEvent.x', 'optaEvent.y']].head())

    optaEvent.timeMin  optaEvent.timeSec optaEvent.playerName  optaEvent.x  \
4                 0.0                0.0         T. Jørgensen         50.1   
17                0.0                4.0           D. Anyembe         28.4   
24                0.0                8.0         K. Jørgensen         20.4   
46                0.0               17.0         K. Jørgensen         39.7   
68                0.0               28.0         K. Jørgensen         73.6   

    optaEvent.y  
4          50.0  
17         43.0  
24         12.8  
46          0.0  
68          0.0  


In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

# 1. Data Loading and Preprocessing
def load_data(match_path):
    """
    Load event data from CSV and preprocess for syncing.
    """
    events_df = pd.read_csv(match_path)

    # Commented out code is only necessary for insufficient opta data

    # Use 2sMarking.startFrameIdx if optaEvent.alignedFrameIdx is missing
    #events_df['frameIdx'] = events_df['optaEvent.alignedFrameIdx'].combine_first(
    #    events_df['2sMarking.startFrameIdx']
    #)

    # Use aligned clock or start clock as fallback
    events_df['clock_time'] = events_df['optaEvent.alignedClock']#.combine_first(
        #events_df['2sMarking.startClock']
    #)

    return events_df


def calculate_video_times(events_df, kickoff_video_time, fps=50):
    """
    Sync events with the video and calculate video times.
    """
    # Find the kickoff frame and calculate offset (Not actually relevant, realized kickoff always happens at 0.04 so it is negligible)
    #kickoff_row = events_df.loc[
    #(events_df['optaEvent.period'] == 1) & 
    #(events_df['2sMarking.deadBallType'] == 'kick_off')
    #]
    #kickoff_clock = kickoff_row['clock_time']
    kickoff_offset = kickoff_video_time# - kickoff_clock

    # Adjust aligned clocks for second-half events
    first_half_end = events_df.loc[events_df['optaEvent.period'] == 1].iloc[-1]
    second_half_start = events_df.loc[events_df['optaEvent.period'] == 2].iloc[0]

    first_half_end_time = pd.to_datetime(first_half_end['optaEvent.timeStamp'])
    second_half_start_time = pd.to_datetime(second_half_start['optaEvent.timeStamp'])

    time_difference = (second_half_start_time - first_half_end_time).total_seconds()
    events_df.loc[events_df['optaEvent.period'] == 2, 'adjusted_alignedClock'] = (
        events_df['optaEvent.alignedClock'] + time_difference
    )

    # Calculate video times for all events
    events_df['video_time'] = events_df['optaEvent.alignedClock'] + kickoff_offset

    return events_df


# 2. Filtering Events
def filter_events(events_df, type_id=None, player=None, location=None, output_file="filtered_events.csv"):
    """
    Filter events based on type, player, or location.
    """
    filtered_df = events_df

    if type_id is not None:
        filtered_df = filtered_df[filtered_df['optaEvent.typeId'] == type_id]

    if player is not None:
        filtered_df = filtered_df[filtered_df['optaEvent.playerName'] == player]

    if location is not None:
        x_min, x_max, y_min, y_max = location
        filtered_df = filtered_df[
            (filtered_df['optaEvent.x'] >= x_min) &
            (filtered_df['optaEvent.x'] <= x_max) &
            (filtered_df['optaEvent.y'] >= y_min) &
            (filtered_df['optaEvent.y'] <= y_max)
        ]

    filtered_df.to_csv(output_file, index=False)
    print(f"Filtered events saved to {output_file}")
    
    return filtered_df

def process_and_filter_events(match_path, video_path, output_file, type_id=None, player=None, location=None, kickoff_video_time=0.04, fps=50):
 
    print("Loading match data...")
    events_df = load_data(match_path)
    
    print("Calculating video times...")
    events_df = calculate_video_times(events_df, kickoff_video_time, fps)
    
    print("Filtering events...")
    filtered_events = filter_events(events_df, type_id=type_id, player=player, location=location, output_file=output_file)
    
    return filtered_events



In [11]:

class VideoPlayer:
    def __init__(self, video_path, events, event_buffer=5):
        """
        Initialize the video player with the video and filtered events.
        :param video_path: Path to the video file.
        :param events: DataFrame with filtered events.
        :param event_buffer: Time (in seconds) to include before and after the event.
        """
        self.video_path = video_path
        self.events = events
        self.event_buffer = event_buffer
        self.current_event_idx = 0

    def display_event(self):
        """
        Play the current event using FFplay, highlighting the event range.
        """
        event = self.events.iloc[self.current_event_idx]
        video_time = event['video_time']
        start_time = max(video_time - self.event_buffer, 0)
        end_time = min(video_time + self.event_buffer, VideoFileClip(self.video_path).duration)

        print(f"Playing event {self.current_event_idx + 1}/{len(self.events)}: {start_time} to {end_time}")

        # Command to play the video with FFplay
        ffplay_command = (
            f'ffplay -i "{self.video_path}" -ss {start_time} -t {end_time - start_time} '
            f'-vf "drawtext=text=\'Event {self.current_event_idx + 1}\':x=10:y=10:fontsize=24:fontcolor=white:box=1:boxcolor=black@0.5"'
        )
        os.system(ffplay_command)

    def next_event(self):
        """
        Navigate to the next event.
        """
        if self.current_event_idx < len(self.events) - 1:
            self.current_event_idx += 1
        else:
            print("You are already at the last event.")

    def previous_event(self):
        """
        Navigate to the previous event.
        """
        if self.current_event_idx > 0:
            self.current_event_idx -= 1
        else:
            print("You are already at the first event.")

    def tag_event(self, relevant, new_start=None, new_end=None):
        """
        Tag the current event as relevant or not and optionally edit start/end times.
        """
        event = self.events.iloc[self.current_event_idx].copy()
        event['relevant'] = relevant

        # Adjust start/end times if provided
        if new_start is not None:
            event['start_time'] = new_start
        else:
            event['start_time'] = max(event['video_time'] - self.event_buffer, 0)

        if new_end is not None:
            event['end_time'] = new_end
        else:
            event['end_time'] = min(event['video_time'] + self.event_buffer, VideoFileClip(self.video_path).duration)

        self.edited_events.append(event)
        print(f"Tagged event {self.current_event_idx + 1} as {'relevant' if relevant else 'not relevant'}.")

    def save_edited_events(self, output_file):
        """
        Save edited events to a CSV file.
        """
        edited_df = pd.DataFrame(self.edited_events)
        edited_df.to_csv(output_file, index=False)
        print(f"Edited events saved to {output_file}")


In [ ]:
match_csv = Path(git_dir / "events_data.csv")
video_path = Path(data_dir / 'output_video.mp4')
output_file="filtered_events.csv"

# Step 1: Process and filter events
filtered_events = process_and_filter_events(
    match_path=match_csv,
    video_path=video_path,
    output_file=output_file,
    type_id=1,  # Event type
    player="J. Grønning",  # Player filter
    kickoff_video_time=324.5,  # Kickoff time in video
    fps=50
)


# Step 2: Initialize VideoPlayer
player = VideoPlayer(str(video_path), filtered_events)

# Step 3: Use the VideoPlayer
# Play the first event
player.display_event()

# Navigate and tag events
player.next_event()
player.display_event()
player.tag_event(relevant=True, new_start=10.0, new_end=20.0)

# Save tagged and edited events
player.save_edited_events("edited_events.csv")


Loading match data...
Calculating video times...
Filtering events...
Filtered events saved to filtered_events.csv


OSError: MoviePy error: the file /assets/output_video.mp4 could not be found!
Please check that you entered the correct path.

In [ ]:
from dash import Dash, html, Input, Output, State, dcc
import pandas as pd
from dash_player import DashPlayer

# File paths and initial setup
match_csv = Path(git_dir / "events_data.csv")
video_path = Path(data_dir / 'output_video.mp4')
output_file = "filtered_events.csv"

# Step 1: Process and filter events using the existing function
filtered_events = process_and_filter_events(
    match_path=str(match_csv),  # Convert Path to string
    video_path=str(video_path),
    output_file=str(output_file),  # Convert Path to string
    type_id=1,  # Example: Passes
    player="J. Grønning",  # Example: Filter events for J. Grønning
    kickoff_video_time=324.5,
    fps=50
)

# Set video path for DashPlayer
video_path = "/assets/output_video.mp4"

# Add columns for tagging and manual start/end times
filtered_events["tag"] = None
filtered_events["start_time"] = filtered_events["video_time"] - 3
filtered_events["end_time"] = filtered_events["video_time"] + 5

# Create Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Interactive Video Player with Tagging"),
    DashPlayer(
        id="video-player",
        url=video_path,
        controls=True,
        width="100%",
        height="500px",
    ),
    html.Br(),
    html.Div([
        html.Button("Previous Event", id="prev-event-btn"),
        html.Button("Next Event", id="next-event-btn"),
    ]),
    html.Br(),
    html.Div(id="event-details", style={"margin-top": "20px"}),
    html.Br(),
    html.Label("Tag this event:"),
    dcc.Dropdown(
        id="tag-dropdown",
        options=[
            {"label": "Relevant", "value": "relevant"},
            {"label": "Not Relevant", "value": "not_relevant"},
        ],
        placeholder="Select tag",
    ),
    html.Br(),
    html.Button("Set Start Time", id="set-start-btn"),
    html.Button("Set End Time", id="set-end-btn"),
    html.Br(),
    html.Button("Export Tagged Events", id="export-events-btn"),
    dcc.Store(id="current-event-index", data=0),
    dcc.Store(id="filtered-events", data=filtered_events.to_dict("records")),
])

# Callback for navigation, tagging, and exporting
@app.callback(
    [
        Output("current-event-index", "data"),
        Output("event-details", "children"),
        Output("tag-dropdown", "value"),
        Output("video-player", "seekTo"),  # Update video playback position
        Output("filtered-events", "data"),  # Update stored filtered events
    ],
    [
        Input("prev-event-btn", "n_clicks"),
        Input("next-event-btn", "n_clicks"),
        Input("set-start-btn", "n_clicks"),
        Input("set-end-btn", "n_clicks"),
        Input("tag-dropdown", "value"),
        Input("export-events-btn", "n_clicks"),
    ],
    [
        State("video-player", "currentTime"),
        State("current-event-index", "data"),
        State("filtered-events", "data"),
    ],
    prevent_initial_call=True,
)
def update_event_details(prev_clicks, next_clicks, set_start, set_end, tag, export_clicks, current_time, current_index, data):
    # Reconstruct filtered_events DataFrame from stored data
    filtered_events = pd.DataFrame(data)
    ctx = dash.callback_context

    if not ctx.triggered:
        return current_index, "No event selected", None, None, filtered_events.to_dict("records")

    button_id = ctx.triggered[0]["prop_id"].split(".")[0]

    # Handle navigation
    if button_id in ["prev-event-btn", "next-event-btn"]:
        if button_id == "prev-event-btn" and current_index > 0:
            current_index -= 1
        elif button_id == "next-event-btn" and current_index < len(filtered_events) - 1:
            current_index += 1

    # Handle setting start/end time
    if button_id == "set-start-btn":
        filtered_events.at[current_index, "start_time"] = current_time
    elif button_id == "set-end-btn":
        filtered_events.at[current_index, "end_time"] = current_time

    # Handle tagging
    if button_id == "tag-dropdown":
        filtered_events.at[current_index, "tag"] = tag

    # Handle exporting tagged events
    if button_id == "export-events-btn":
        tagged_events = filtered_events.dropna(subset=["tag"])  # Keep only tagged events
        print(tagged_events)  # Debug: Show tagged events in console
        if not tagged_events.empty:
            tagged_events.to_csv("tagged_events.csv", index=False)
            export_message = f"Exported {len(tagged_events)} tagged events to 'tagged_events.csv'."
        else:
            export_message = "No tagged events to export."
        return current_index, export_message, tag, None, filtered_events.to_dict("records")

    # Update event details and seekTo property
    event = filtered_events.iloc[current_index]
    details = f"Event {current_index + 1}: Start {event['start_time']}s, End {event['end_time']}s, Tag: {event['tag']}"
    seek_time = max(event["video_time"] - 3, 0)  # Seek to 3 seconds before event

    return current_index, details, event["tag"], seek_time, filtered_events.to_dict("records")


if __name__ == "__main__":
    app.run_server(debug=True)


Loading match data...
Calculating video times...
Filtering events...
Filtered events saved to filtered_events.csv


   optaEvent.id  optaEvent.seqId  optaEvent.eventId  optaEvent.typeId  \
3    2751211171             1982                123                 1   
7    2751217717              798                224                 1   

   optaEvent.periodId  optaEvent.timeMin  optaEvent.timeSec  \
3                   1                 12                  5   
7                   1                 20                 26   

      optaEvent.contestantId  optaEvent.opContestantId  optaEvent.opPlayerId  \
3  cftj6g399tknheudaxsdpfdgb                       547                165922   
7  cftj6g399tknheudaxsdpfdgb                       547                165922   

   ...  2sMarking.nextContactLoc  2sMarking.secondContactId  \
3  ...                      None                       None   
7  ...                      None                       None   

   2sMarking.isOffenseNextContact 2sMarking.cardType   clock_time  \
3                            None               None   722.799988   
7                    

In [37]:
print("Dash version:", dash.__version__)

Dash version: 2.18.2


In [ ]:
# 3. Clip Extraction
def extract_clips(events_df, video_path, output_dir, event_buffer=5):
    """
    Extract clips for each event in the DataFrame and save them as separate video files.
    Includes debugging output for each clip.
    """
    # Ensure output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Drop rows with NaN in 'video_time' to avoid errors
    events_df = events_df.dropna(subset=['video_time'])

    for idx, row in events_df.iterrows():
        start_time = max(row['video_time'] - event_buffer, 0)
        end_time = row['video_time'] + event_buffer
        output_path = str(Path(output_dir) / f"event_clip_{idx}.mp4")

        # Debug output for each event
        print(f"Processing event {idx}: Start={start_time}, End={end_time}, Output={output_path}")

        try:
            # Convert video_path to string if it's a Path object
            with VideoFileClip(str(video_path)) as video:
                clip = video.subclip(start_time, end_time)  # Create subclip
                clip.write_videofile(output_path, codec="libx264")  # Save the clip
                print(f"Saved clip to {output_path}")
        except Exception as e:
            print(f"Error processing clip for event {idx}: {e}")


# 4. Main Function
def process_match(match_csv, video_path, output_dir, kickoff_video_time):
    """
    Complete pipeline for processing a match and extracting clips.
    """
    # Load and preprocess data
    events_df = load_data(match_csv)
    events_df = calculate_video_times(events_df, kickoff_video_time)

    # Filter for passes (example)
    passes_df = filter_events(events_df, type_id=1, player="J. Grønning")

    # Extract clips
    extract_clips(passes_df, video_path, output_dir)


# Example Usage
#if __name__ == "__main__":
    #match_csv = "bachelor_vff/events_data.csv"  # Path to event data
    #video_path = "bachelor_vff/match_video.mp4"  # Path to video
    #output_dir = "bachelor_vff/output_clips"     # Directory for output clips
    #kickoff_video_time = 120.0  # Example kickoff time in video (seconds)

    #process_match(match_csv, video_path, output_dir, kickoff_video_time)


In [37]:
match_csv = Path(git_dir / "events_data.csv")
video_path = Path(data_dir / 'output_video.mp4')
output_dir = Path(git_dir/ 'output_clips')
kickoff_video_time = 324.5

process_match(match_csv, video_path, output_dir, kickoff_video_time)

Processing event 241: Start=470.86000061035156, End=480.86000061035156, Output=c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_241.mp4
Moviepy - Building video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_241.mp4.
MoviePy - Writing audio in event_clip_241TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_241.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_241.mp4
Saved clip to c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_241.mp4
Processing event 352: Start=533.9799957275391, End=543.9799957275391, Output=c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_352.mp4
Moviepy - Building video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_352.mp4.
MoviePy - Writing audio in event_clip_352TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_352.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_352.mp4
Saved clip to c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_352.mp4
Processing event 508: Start=624.6199951171875, End=634.6199951171875, Output=c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_508.mp4
Moviepy - Building video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_508.mp4.
MoviePy - Writing audio in event_clip_508TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7510.mp4
Saved clip to c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7510.mp4
Processing event 7562: Start=2970.4599609375, End=2980.4599609375, Output=c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7562.mp4
Moviepy - Building video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7562.mp4.
MoviePy - Writing audio in event_clip_7562TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7562.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7562.mp4
Saved clip to c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7562.mp4
Processing event 7614: Start=3000.4599609375, End=3010.4599609375, Output=c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7614.mp4
Moviepy - Building video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7614.mp4.
MoviePy - Writing audio in event_clip_7614TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7614.mp4



Moviepy - Done !
Moviepy - video ready c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7614.mp4
Saved clip to c:\Users\viggo\Documents\Cognitive Science\5th semester\Bachelor\git_workspace\bachelor_vff\output_clips\event_clip_7614.mp4
